# Dataset analysis



In [1]:
!pip install datasets  # huggingface library with dataset
!pip install conllu    # aux library for processing CoNLL-U format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import time
import random
import torch
import torch.nn as nn
import numpy as np
from functools import partial
from datasets import load_dataset

# Description of baseline and BERT model

## Arc-eager

A configuration of the arc-eager parser is a triple of the form $(σ, β, A)$ where:

* $\sigma$ is the stack
* $\beta$ is the buffer
* $A$ is the set of arcs constructed so far

Let:

* $\beta_i$, $i\geq1$ the $i$-th token in the buffer
* $\sigma_i$, $i\geq1$ the $i$-th token in the stack

for the $i$-th configuration

The arc-eager parser can perform four types of actions (transitions):

* **left-arc** (LA): create the arc $(\beta_1 → \sigma_1)$ and remove $\sigma_1$ from the stack. The **preconditions** are: $\sigma_1$ is not the ROOT and $\sigma_1$ does not have already an head

* **right-arc** (RA): create the arc $(\sigma_1 → \beta_1)$ and push $\beta_1$ to the stack

* **reduce** (RE): remove $\sigma_1$ from the stack. The **precondition** is: $\sigma_1$ must have a head

* **shift** (SH): remove $\beta_1$ from the buffer and push it to the stack


Let $w = w_0 w_1 \cdots w_{n}$ be the input sentence, with $w_0$ the special symbol `<ROOT>`.
Stack and buffer are implemented as lists of integers, where `j` represents word $w_j$.  Top-most stack token is at the right-end of the list; first buffer token is at the left-end of the list. 
Set $A$ is implemented as an array `arcs` of size $n+1$ such that if arc $(w_i \rightarrow w_j)$ is in $A$ then `arcs[j]=i`, and if $w_j$ is still missing its head node in the tree under construction, then `arcs[j]=-1`. We always have `arcs[0]=-1`.  We use this representation also for complete dependency trees.

In [3]:
# NOTE: here only the transition methods are implemented
# the preconditions will be checked later with the boolean methods

class ArcEager:
  def __init__(self, sentence):
    self.sentence = sentence
    self.buffer = [i for i in range(len(self.sentence))]  # buffer
    self.stack = []   # empty stack
    self.arcs = [-1 for _ in range(len(self.sentence))]  # non-valid arcs

    # initialization of the stack with one SH operations
    # only the <ROOT> in the stack
    self.shift()

  # left arc
  def left_arc(self):
    b1 = self.buffer[0]
    o1 = self.stack.pop()
    self.arcs[o1] = b1

  # right arc
  def right_arc(self):
    b1 = self.buffer[0]
    o1 = self.stack[-1]
    self.arcs[b1] = o1
    self.buffer = self.buffer[1:]
    self.stack.append(b1)
    
  # reduce
  def reduce(self):
    _ = self.stack.pop()

  # shift
  def shift(self):
    b1 = self.buffer[0]
    self.buffer = self.buffer[1:]
    self.stack.append(b1)

  # print configuration (for debug)
  def print_configuration(self):
    s = [self.sentence[i] for i in self.stack]
    b = [self.sentence[i] for i in self.buffer]
    print(s, b)
    print(self.arcs) 

  def is_tree_final(self):
    return len(self.buffer) == 0
    

## Oracle

The transition actions must follow the preconditions above

In [4]:
class Oracle:
  def __init__(self, parser, gold_tree):
    self.parser = parser
    self.gold = gold_tree

  # left arc?
  def is_left_arc_gold(self):
    # get the two elems the arc should be built on
    b1 = self.parser.buffer[0]
    o1 = self.parser.stack[len(self.parser.stack)-1]

    # check preconditions
    if o1 == -1: return False
    if self.parser.arcs[o1] != -1: return False

    # check if gold move
    if self.gold[o1] != b1:
      return False
    else:
      return True

  # right arc?
  def is_right_arc_gold(self):
    b1 = self.parser.buffer[0]
    o1 = self.parser.stack[len(self.parser.stack)-1]

    # no preconditions?
    # check if gold move
    if self.gold[b1] != o1:
      return False
    else:
      return True

  # reduce?
  def is_reduce_gold(self):

    # precondition
    o1 = self.parser.stack[len(self.parser.stack)-1]
    if self.parser.arcs[o1] == -1:
      return False
    
    # check if exist k < o1 s.t. exist (k,b1) or (b1,k) in Agold
    b1 = self.parser.buffer[0]
    if self.gold[b1] < o1:
      return True
    else:
      return b1 in self.gold[:o1]
    

  


## Test on a sentence

In [5]:
# define the sentence and the gold tree
sentence = ["<ROOT>", "He", "wrote", "her", "a", "letter", "."]
gold = [-1, 2, 0, 2, 5, 2, 2]

# initialize parser and oracle
parser = ArcEager(sentence)
oracle = Oracle(parser, gold)

# print initial configuration
parser.print_configuration()

['<ROOT>'] ['He', 'wrote', 'her', 'a', 'letter', '.']
[-1, -1, -1, -1, -1, -1, -1]


In [6]:
# until the tree is final, apply the right move

iteration = 0   # keep track of the number of iterations
while not parser.is_tree_final():
  print('Iteration:',iteration)

  # if LA is gold
  if oracle.is_left_arc_gold():
    parser.left_arc()
    parser.print_configuration()
    print('Transition: LA', end='\n\n')

  # elif RA is gold
  elif oracle.is_right_arc_gold():
    parser.right_arc()
    parser.print_configuration()
    print('Transition: RA', end='\n\n')

  # elif RE is gold
  elif oracle.is_reduce_gold():
    parser.reduce()
    parser.print_configuration()
    print('Transition: RE', end='\n\n')

  # else shift
  else:
    parser.shift()
    parser.print_configuration()
    print('Transition: SH', end='\n\n')


  iteration = iteration + 1

# parsing tree completed

Iteration: 0
['<ROOT>', 'He'] ['wrote', 'her', 'a', 'letter', '.']
[-1, -1, -1, -1, -1, -1, -1]
Transition: SH

Iteration: 1
['<ROOT>'] ['wrote', 'her', 'a', 'letter', '.']
[-1, 2, -1, -1, -1, -1, -1]
Transition: LA

Iteration: 2
['<ROOT>', 'wrote'] ['her', 'a', 'letter', '.']
[-1, 2, 0, -1, -1, -1, -1]
Transition: RA

Iteration: 3
['<ROOT>', 'wrote', 'her'] ['a', 'letter', '.']
[-1, 2, 0, 2, -1, -1, -1]
Transition: RA

Iteration: 4
['<ROOT>', 'wrote', 'her', 'a'] ['letter', '.']
[-1, 2, 0, 2, -1, -1, -1]
Transition: SH

Iteration: 5
['<ROOT>', 'wrote', 'her'] ['letter', '.']
[-1, 2, 0, 2, 5, -1, -1]
Transition: LA

Iteration: 6
['<ROOT>', 'wrote'] ['letter', '.']
[-1, 2, 0, 2, 5, -1, -1]
Transition: RE

Iteration: 7
['<ROOT>', 'wrote', 'letter'] ['.']
[-1, 2, 0, 2, 5, 2, -1]
Transition: RA

Iteration: 8
['<ROOT>', 'wrote'] ['.']
[-1, 2, 0, 2, 5, 2, -1]
Transition: RE

Iteration: 9
['<ROOT>', 'wrote', '.'] []
[-1, 2, 0, 2, 5, 2, 2]
Transition: RA



In [7]:
# check the arcs obtained are the same as in the gold tree
print('Is tree correct?', oracle.gold == parser.arcs)

Is tree correct? True


# Data set-up and training

## Dataset

## Preprocessing

### Utils for parsing

In [8]:
# function to check if the tree is projective or not
def is_projective(tree):
  for i in range(len(tree)):
    if tree[i] == -1:
      continue
    left = min(i, tree[i])
    right = max(i, tree[i])

    for j in range(0, left):
      if tree[j] > left and tree[j] < right:
        return False
    for j in range(left+1, right):
      if tree[j] < left or tree[j] > right:
        return False
    for j in range(right+1, len(tree)):
      if tree[j] > left and tree[j] < right:
        return False

  return True

def parse_sentence(oracle, show_conf=False):
    if show_conf:
        oracle.parser.print_configuration()
    iteration = 0   # keep track of the number of iterations
    while not oracle.parser.is_tree_final():
        if show_conf:
            print('Iteration:',iteration)

        # if LA is gold
        if oracle.is_left_arc_gold():
            oracle.parser.left_arc()
            if show_conf:
                oracle.parser.print_configuration()
                print('Transition: LA', end='\n\n')

        # elif RA is gold
        elif oracle.is_right_arc_gold():
            oracle.parser.right_arc()
            if show_conf:
                oracle.parser.print_configuration()
                print('Transition: RA', end='\n\n')

        # elif RE is gold
        elif oracle.is_reduce_gold():
            oracle.parser.reduce()
            if show_conf:
                oracle.parser.print_configuration()
                print('Transition: RE', end='\n\n')

        # else shift
        else:
            oracle.parser.shift()
            if show_conf:
                oracle.parser.print_configuration()
                print('Transition: SH', end='\n\n')


        iteration = iteration + 1
    return oracle.parser.arcs

# get a random sentence to test the parsing algorith to be correct
def parse_dataset(dataset):
    for sample in dataset:
        sentence = ['<ROOT>'] + sample['tokens']
        gold_tree = sample['head']
        gold_tree = [-1] + [int(key) for key in gold_tree]
        # initialize parser and oracle
        parser = ArcEager(sentence)
        oracle = Oracle(parser, gold_tree)
        parse_tree = parse_sentence(oracle)
        if parse_tree != gold_tree:
            return False
    return True

### Test the oracle and the parsing functions on the training set

In [9]:
# get english sentences from the dataset
train_dataset = load_dataset('universal_dependencies', 'en_lines', split="train")
# remove non-projective sentences: heads in the gold tree are strings, we convert them to int
train_dataset = [sample for sample in train_dataset if is_projective([-1] + [int(head) for head in sample["head"]])]

# parse the whole dataset $train_dataset using the oracle to check if it works correctly
print(parse_dataset(train_dataset))

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1032 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1035 [00:00<?, ? examples/s]

Dataset universal_dependencies downloaded and prepared to /root/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7. Subsequent calls will reuse this data.
True


### Pre-processing utils
- the $create\_token\_indices$ function creates a vocabulary $vocab$ containing all tokens in $dataset$ (given as input argument) appearing at least $threshold$ (given as input argument, default value 3) times. $vocab[token]$ is the index assigned to $token;

- the $process\_sample$ function is used to process our data and create the actual training samples.
For each sentence in $train\_dataset$, we use our oracle to compute the canonical path followed to extract the gold tree. We then pair each configuration to the golden transition selected by the oracle.
Because of the structure of Arc-Eager parser, we encode a $configuration$ with only two words: $\sigma_1$ and $\beta_1$ (i.e. the topmost element on the stack and the first buffer element, respectiveley);

- $prepare\_batch$ function pre-processes a batch of samples $batch\_data$ using as indices of tokens the ones contained in the vocabulary $tokens\_indices\_voc$. The pre-processing is done by applying function $process\_sample$ to each sample in $batch\_data$.

In [10]:
# the function returns a dictionary containing the vocabulary embedding indices for the tokens in $dataset
# i.e. an index is associated to each token in $dataset
# $threshold is the minimum number of appearance for a token in $dataset to be included in the dictionary
def create_token_indices(dataset, threshold=3):
  # $dic has the tokens as keys. Given a token, $dic[token] is the number of occurrences of $token in $dataset
  dic = {}
  for sample in dataset:
    for token in sample['tokens']:
      if token in dic:
        dic[token] += 1
      else:
        dic[token] = 1 

  # vocab["token"] is an integer representing the index of token "token"
  vocab = {}
  # indices for some special tokens
  vocab["<pad>"] = 0
  vocab["<ROOT>"] = 1 
  vocab["<unk>"] = 2

  next_ind = 3
  for token in dic.keys():
    if dic[token] >= threshold:
      vocab[token] = next_ind
      next_ind += 1

  return vocab

# creates training instances from one sample if $get_gold_path is $True, otherwise does only a simple pre-process
# $sample is a sample of our dataset
def process_sample(tokens_indices_voc, sample, get_gold_path = False):

  # add the root token to the sentence and its head (-1) to the gold head list
  sentence = ["<ROOT>"] + sample["tokens"]
  gold = [-1] + [int(i) for i in sample["head"]]
  
  # sentence representation with each token represented by its index in the vocabulary
  sentence_repr = [tokens_indices_voc[token] if token in token_indices_voc else token_indices_voc["<unk>"] for token in sentence]

  # $gold_configurations and $gold_transitions are parallel arrays whose elements refer to parsing steps
  # $gold_configurations[i] records configuration at step $i, i.e. topmost stack token and first buffer token for current step
  gold_configurations = []
  # $gold_transitions[i] contains oracle (canonical) transition for step $i: 0 is left_arc, 1 right_arc, 2 reduce, 3 shift
  gold_transitions = []

  # only for training
  if get_gold_path:
    parser = ArcEager(sentence)
    oracle = Oracle(parser, gold)

    while not parser.is_tree_final():
      
      # save configuration
      configuration = [parser.stack[-1]]
      if len(parser.buffer) == 0:
        configuration.append(-1)
      else:
        configuration.append(parser.buffer[0])  
      gold_configurations.append(configuration)

      # save gold transition
      if oracle.is_left_arc_gold():
        gold_transitions.append(0)
        oracle.parser.left_arc()
      elif oracle.is_right_arc_gold():
        gold_transitions.append(1)
        oracle.parser.right_arc()
      elif oracle.is_reduce_gold():
        gold_transitions.append(2)
        oracle.parser.reduce()
      else:
        gold_transitions.append(3)
        oracle.parser.shift()

  # $sentence_repr is a list containing representations of tokens in $sample
  # $gold is a list containning the representation of the gold tree of $sample
  # $gold_configurations is a list containing gold configurations
  # $gold_transitions is a list containing gold transitions
  return sentence_repr, gold_configurations, gold_transitions, gold

# this function is used to pre-process a batch of samples $batch_data from the original dataset
# applying function $process_sample to each sample in $batch_data
def prepare_batch(tokens_indices_voc, batch_data, get_gold_path=False):
  processed_batch_data = [process_sample(token_indices_voc, sample, get_gold_path=get_gold_path) for sample in batch_data]
  
  sentences_repr, paths, moves, gold_trees = []
  for sample in processed_batch_data:
    sentences_repr.append(sample[0])
    paths.append(sample[1])
    moves.append(sample[2])
    gold_trees.append(sample[3])
    
    # sentences_repr, paths, moves, gold_trees are parallel lists
    # element in position $i of each of the above lists refers to the same sentence $i 
  return sentences_repr, paths, moves, gold_trees

### Datasets loading and pre-processing

In [11]:
# the training set has already been loaded, then load also development set and test set
dev_dataset = load_dataset('universal_dependencies', 'en_lines', split="validation")
test_dataset = load_dataset('universal_dependencies', 'en_lines', split="test")

# set the number of samples per batch
BATCH_SIZE = 32

# create the dictionary with token indices 
tokens_indices = create_token_indices(train_dataset)

# create dataloaders to batch each dataset and apply function $prepare_batch to each batch
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=partial(prepare_batch, tokens_indices, get_gold_path=True))
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=partial(prepare_batch))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=partial(prepare_batch))

#### Now everything is ready to create a BiLSTM and extract embeddings from configurations in order to then train a classifier based on gold trees

### BERT model
In this section, we will use BERT to extract a feature vector for each token $\sigma_2$, $\sigma_1$, $\beta_1$ in $configuration$.

In [12]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 21.0 MB/s eta 0:00:00


In [13]:
from transformers import AutoTokenizer, AutoModel
 
model_name = "bert-base-uncased"

model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


For each sample in the dataset, we take the corresponding sentence and we tokenize it, assigning an embedding to each token. The tokenizer uses BPE, so some words might be split into several tokens. Since we want to assign an embedding to three words ($\sigma_2$, $\sigma_1$, $\beta_1$), if a word in the sentence is split into more than one token, we average the BPE of those token, obtaining the embeddigng of the entire word.
We noticed that, in case of split word into several tokens, the structure is always:
$"word" = "sub\_word_1", "\#\#sub\_word_2", ..., "\#\#sub\_word_n"$
Thus, we can iterate while we find a subsequent word that starts with $"\#\#"$.

['[CLS]', 'eating', 'chips', 'is', 'beautiful', '[SEP]']

# Evaluation

# SotA discussion